In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/generative-dog-images/Annotation.zip
/kaggle/input/generative-dog-images/all-dogs.zip


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from scipy.linalg import sqrtm
from tqdm import tqdm

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define paths
dataset_path = '/kaggle/input/generative-dog-images'
output_path = '/kaggle/working/'


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Define hyperparameters
latent_dim = 100
n_epochs = 200
batch_size = 32

In [16]:
import os

dataset_path = '/kaggle/input/generative-dog-images/all-dogs.zip'  # Modify this path according to your dataset location

files = os.listdir(dataset_path)
print(files)


NotADirectoryError: [Errno 20] Not a directory: '/kaggle/input/generative-dog-images/all-dogs.zip'

In [15]:
# Preprocess the image
def preprocess_image(image_path):
    image = tf.io.read_file(tf.cast(image_path, tf.string))
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.float32)
    image = (image / 255.0) * 2 - 1
    return image

# Load and preprocess the dataset
dataset = tf.data.Dataset.list_files(os.path.join(dataset_path, '*.jpg'))
dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)


InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: /kaggle/input/generative-dog-images/*.jpg'

In [4]:
# Load and preprocess the dog image dataset
def load_dataset():
    image_paths = []
    
    for root, _, filenames in os.walk(dataset_path):
        for filename in filenames:
            if filename.endswith('.jpg'):
                image_path = os.path.join(root, filename)
                image_paths.append(image_path)
    
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    
    def preprocess_image(image_path):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (64, 64))
        image = image / 255.0  # Normalize pixel values to [0, 1]
        return image
    
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)
    
    return dataset


In [5]:
# Define the generator model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8 * 8 * 256, input_shape=(latent_dim,)),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, kernel_size=(4, 4), strides=(2, 2), padding='same', activation='relu'),
        layers.Conv2DTranspose(64, kernel_size=(4, 4), strides=(2, 2), padding='same', activation='relu'),
        layers.Conv2DTranspose(3, kernel_size=(4, 4), strides=(2, 2), padding='same', activation='sigmoid')
    ])
    return model

In [6]:
# Define the discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, kernel_size=(4, 4), strides=(2, 2), padding='same', input_shape=(64, 64, 3)),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=(4, 4), strides=(2, 2), padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, kernel_size=(4, 4), strides=(2, 2), padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

In [7]:
# Define the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model

In [8]:
# Generate a batch of random latent vectors
def generate_latent_points(n_samples):
    return np.random.normal(size=(n_samples, latent_dim))

In [9]:
# Generate fake samples with the generator
def generate_fake_samples(generator, n_samples):
    latent_points = generate_latent_points(n_samples)
    fake_samples = generator.predict(latent_points)
    return fake_samples

In [10]:
# Calculate the FID (Fréchet Inception Distance) between real and generated samples
def calculate_fid(real_samples, generated_samples):
    # Preprocess the images for InceptionV3 model
    real_samples = (real_samples * 255).astype(np.uint8)
    generated_samples = (generated_samples * 255).astype(np.uint8)
    
    # Load pre-trained InceptionV3 model
    inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    
    # Compute activations for real and generated samples
    real_activations = inception_model.predict(real_samples)
    generated_activations = inception_model.predict(generated_samples)
    
    # Calculate the mean and covariance of real and generated sample activations
    mu1, sigma1 = np.mean(real_activations, axis=0), np.cov(real_activations, rowvar=False)
    mu2, sigma2 = np.mean(generated_activations, axis=0), np.cov(generated_activations, rowvar=False)
    
    # Compute the squared Euclidean distance between means and trace of covariance product
    fid_score = np.sum((mu1 - mu2) ** 2) + np.trace(sigma1 + sigma2 - 2 * sqrtm(np.dot(sigma1, sigma2)))
    
    return fid_score

In [11]:
# Train the GAN model
def train_gan(gan, dataset, n_epochs, batch_size):
    generator, discriminator = gan.layers
    n_batches = len(dataset) // batch_size
    
    for epoch in range(n_epochs):
        for batch_idx, real_samples in tqdm(enumerate(dataset), total=n_batches, desc=f'Epoch {epoch+1}/{n_epochs}'):
            # Train discriminator
            noise = generate_latent_points(batch_size)
            fake_samples = generator.predict(noise)
            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))
            
            discriminator_loss_real = discriminator.train_on_batch(real_samples, real_labels)
            discriminator_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)
            discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
            
            # Train generator
            noise = generate_latent_points(batch_size)
            generator_loss = gan.train_on_batch(noise, real_labels)
        
        # Evaluate the GAN model using MiFID metric
        real_samples = np.concatenate([sample for sample in dataset], axis=0)
        generated_samples = generate_fake_samples(generator, len(real_samples))
        fid_score = calculate_fid(real_samples, generated_samples)
        
        print(f"Epoch {epoch+1}/{n_epochs}, Generator Loss: {generator_loss:.4f}, Discriminator Loss: {discriminator_loss:.4f}, MiFID: {fid_score:.4f}")
        
        # Save generated samples every 10 epochs
        if (epoch + 1) % 10 == 0:
            generated_samples = generate_fake_samples(generator, 100)
            save_generated_samples(generated_samples, epoch + 1)
    
    # Save the generator model
    generator.save(os.path.join(output_path, 'generator_model.h5'))


In [12]:
# Save generated samples as images
def save_generated_samples(samples, epoch):
    os.makedirs(output_path, exist_ok=True)
    
    for i, sample in enumerate(samples):
        image_path = os.path.join(output_path, f'generated_sample_epoch{epoch}_sample{i+1}.jpg')
        sample = (sample * 255).astype(np.uint8)
        tf.keras.preprocessing.image.save_img(image_path, sample)

In [13]:
# Load and preprocess the dataset
dataset = load_dataset()

TypeError: in user code:

    File "/tmp/ipykernel_32/2752601170.py", line 14, in preprocess_image  *
        image = tf.io.read_file(image_path)

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.
